# Image Classification Using CNN with VGG19 Architecture !

### 1 Input Fundus Image

In [4]:
import os

import random
import scipy.io
import pandas as pd

import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.manifold import TSNE
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
import shutil

In [5]:
import os
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.utils import load_img
from keras.preprocessing.image import ImageDataGenerator
import random
import scipy.io
import pandas as pd

import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.manifold import TSNE
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import shutil

### Input param

In [ ]:
param = {
    'root_dir': './data',
    'RWROP_data_dir': 'RWROPclassimage',
    'RWROP_categories': ['RWROP_0', 'RWROP_1'],
    'CLAHE_clipLimit': 10,
}

### Create image_dir for training and testing the model

In [ ]:
root_dir = param['root_dir']
RWROP_dir_categories = ['Aug_RWROP_ROSE','RWROP_ROSE', 'Aug_RWROP_trainset','RWROP_traindata',
                        'RWROP_trainset', 'RWROP_valset','RWROP_testset']
RWROP_categories = param['RWROP_categories']
for i in range(5):
    fold_dir = os.path.join(root_dir, 'fold_'+ str(i))
    for name in RWROP_dir_categories:
        data_dir = os.path.join(fold_dir, name)
        try:
            shutil.rmtree(data_dir)
        except Exception as e:
            print('create dir error')
            pass
        for category in RWROP_categories:
            path = os.path.join(data_dir, category)
            os.makedirs(path)

### CLAHE

In [ ]:
clahe = cv2.createCLAHE(clipLimit=param['CLAHE_clipLimit'])
RWROP_categories = param['RWROP_categories']
data_dir = param['RWROP_data_dir']
clahe_dir = os.path.join(param['root_dir'], RWROP_data_dir+'_clahe')

for category in RWROP_categories:
    path = os.path.join(data_dir, category)
    clahe_path = os.path.join(clahe_dir, category)
    os.makedirs(clahe_path)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path,img))
            img_green_channel = img_array[: , : , 1]
            fit_img_green_channel = clahe.apply(img_green_channel)+1
            img_array[: , : , 1] = fit_img_green_channel
            cv2.imwrite(os.path.join(clahe_path, img[:-4]+'_clahe.png'), img_array)
        except Exception as e:
            print('clahe error on following img')
            print(os.path.join(path,img))

### 5-fold dataset

In [ ]:
RWROP_data = []
RWROP_label = []
RWROP_categories = param['RWROP_categories']
RWROP_data_dir = os.path.join(param['root_dir'], RWROP_data_dir+'_clahe')

for category in RWROP_categories:
    path = os.path.join(RWROP_data_dir, category)
    class_num = RWROP_categories.index(category)
    for img in os.listdir(path):
        try: 
            RWROP_data.append(os.path.join(path, img))
            RWROP_label.append(class_num)
        except Exception as e:
            print("error in labeling")
            pass
print('\n RWROP dataset size: {0}'.format(len(RWROP_data)))
check_RWROP_label = set(RWROP_label)
print(check_RWROP_label)






In [ ]:
kflod_dir = param['root_dir']
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf.get_n_splits(RWROP_data, RWROP_label)
for i, (train_index, test_index) in enumerate(skf.split(RWROP_data, RWROP_label)):
    for element in train_index:
        img_array = cv2.imread(RWROP_data[element])
        cv2.imwrite(os.path.join(kflod_dir, 'fold_'+ str(i), 'RWROP_traindata', RWROP_categories[RWROP_label[element]], str(RWROP_label[element]) + '_' + os.path.basename(RWROP_data[element])), img_array)
                      
    for element in test_index:
        img_array = cv2.imread(RWROP_data[element])
        cv2.imwrite(os.path.join(kflod_dir, 'fold_'+ str(i), 'RWROP_testset', RWROP_categories[RWROP_label[element]], str(RWROP_label[element]) + '_' + os.path.basename(RWROP_data[element])), img_array)



### Traindata split to trainset and valset

In [ ]:
val_ratio = 0.25
root_dir = param['root_dir']
RWROP_categories = param['RWROP_categories']

for i in range(5):
    fold_dir = os.path.join(root_dir, 'fold_'+ str(i),'RWROP_traindata')
    data = []
    label = []
    for category in RWROP_categories:
        path = os.path.join(fold_dir, category)
        class_num = stage_categories.index(category)
        for img in os.listdir(path):
            try: 
                data.append(os.path.join(path, img))
                label.append(class_num)
            except Exception as e:
                print("error in labeling")
                pass
    X_train, X_val, y_train, y_val = train_test_split(data, label, test_size = val_ratio, random_state = 42)
    for idx in range(X_train.shape[0]):
        img_array = cv2.imread(X_train[idx])
        cv2.imwrite(os.path.join(root_dir, 'fold_'+ str(i), 'RWROP_trainset', RWROP_categories[y_train[idx]],os.path.basename(X_train[idx])), img_array)
    for idx in range(X_val.shape[0]):
        img_array = cv2.imread(X_val[idx])
        cv2.imwrite(os.path.join(root_dir, 'fold_'+ str(i), 'RWROP_valset', RWROP_categories[y_val[idx]],os.path.basename(X_val[idx])), img_array)
            


### ROSE Oversampling

In [ ]:
root_dir = param['root_dir']
RWROP_categories = param['RWROP_categories']

for i in range(5):
    RWROP_data = []
    RWROP_label = []
    fold_dir = os.path.join(root_dir, 'fold_'+ str(i), 'RWROP_trainset')
    for category in RWROP_categories:
        path = os.path.join(fold_dir, category)
        class_num = RWROP_categories.index(category)
        for img in os.listdir(path):
            try: 
                img_array = cv2.imread(os.path.join(path,img))
                new_array = img_array.flatten()
                RWROP_data.append(new_array)
                RWROP_label.append(class_num)
            except Exception as e:
                print("error in labeling")
                pass
    RWROP_data = np.array(RWROP_data)
    RWROP_label = np.array(RWROP_label)
    rose = RandomOverSampler(random_state=42)
    rose_data_resampled, rose_label_resampled = rose.fit_resample(RWROP_data, RWROP_label)
    ROSE_data_dir = os.path.join(root_dir, 'fold_'+ str(i), 'RWROP_ROSE')
    for idx in range(rose_data_resampled.shape[0]):
        cv2.imwrite(os.path.join(ROSE_data_dir, RWROP_categories[rose_label_resampled[idx]], str(rose_label_resampled[idx])+'_ROSE_' + str(idx) + '.png'), rose_data_resampled[idx].reshape(480,640,3))    


### Data Augmentation 

In [7]:
kflod_dir = param['root_dir']
RWROP_categories = ['RWROP_0', 'RWROP_1']
datagen = ImageDataGenerator(
        rotation_range= 30,
        zoom_range = [0.9, 1.1],
        horizontal_flip = True,
        fill_mode ='constant',
        cval= 0)


In [ ]:
for category in RWROP_categories:
    for k in range(5):
        load_dir = os.path.join(kflod_dir, 'fold_'+ str(k), 'RWROP_ROSE', category)
        for img in os.listdir(load_dir):
            image = load_img(load_dir+'/{0}'.format(img)) 
            x = img_to_array(image)
            x = x.reshape((1,) + x.shape)
            i = 0
            for batch in datagen.flow(x, batch_size=1,
                                       save_to_dir=os.path.join(kflod_dir, 'fold_'+ str(k), 'Aug_RWROP_ROSE', category), 
                                       save_prefix=category, save_format='png'):
                i += 1
                if i > 2:
                    break

In [ ]:
for k in range(5):
    load_dir = os.path.join(kflod_dir, 'fold_'+ str(k), 'RWROP_trainset', 'RWROP_0')
    for img in os.listdir(load_dir):
        image = load_img(load_dir+'/{0}'.format(img)) 
        x = img_to_array(image)  
        x = x.reshape((1,) + x.shape)

        i = 0
        for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=os.path.join(kflod_dir, 'fold_'+ str(k), 'Aug_RWROP_trainset', 'RWROP_0'), 
                              save_prefix='RWROP_0', save_format='png'):
            i += 1
            if i > 2:
                break

In [ ]:
for k in range(5):
    load_dir = os.path.join(kflod_dir, 'fold_'+ str(k), 'RWROP_trainset', 'RWROP_1')
    for img in os.listdir(load_dir):
        image = load_img(load_dir+'/{0}'.format(img)) 
        x = img_to_array(image)  
        x = x.reshape((1,) + x.shape)

        i = 0
        for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=os.path.join(kflod_dir, 'fold_'+ str(k), 'Aug_RWROP_trainset', 'RWROP_1'), 
                              save_prefix='RWROP_1', save_format='png'):
            i += 1
            if i > 35:
                break